In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

# 5.3 Lab: Cross-Validation and the Bootstrap

**5.3.1 The Validation Set Approach**

In [3]:
Auto = pd.read_csv('Data/Auto.csv', na_values='?').dropna()
Auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


> library(ISLR)

> set.seed(1)

> train=sample (392,196)

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(Auto, test_size = 0.5, random_state = 1)

In [5]:
#> lm.fit=lm(mpg∼horsepower ,data=Auto,subset=train)
y_train,X_train = patsy.dmatrices('mpg ~ 0 + horsepower',data = train_set,return_type="dataframe")
y_test,X_test = patsy.dmatrices('mpg ~ 0 + horsepower',data = test_set,return_type="dataframe")
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

> attach(Auto)

> mean((mpg -predict(lm.fit ,Auto))[-train]^2)

In [6]:
((y_test - lin_reg.predict(X_test))**2).mean()

mpg    24.802121
dtype: float64

In [7]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,lin_reg.predict(X_test))

24.80212062059356

> lm.fit2=lm(mpg∼poly(horsepower ,2),data=Auto,subset=train)

> mean((mpg -predict(lm.fit2,Auto))[-train]^2)

> lm.fit3=lm(mpg∼poly(horsepower ,3),data=Auto,subset=train)

> mean((mpg -predict(lm.fit3,Auto))[-train]^2)


In [8]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree = 2,include_bias=False)
X_train_degree = poly_features.fit_transform(X_train)
X_test_degree =  poly_features.fit_transform(X_test)
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train_degree,y_train)
mean_squared_error(y_test,lin_reg.predict(X_test_degree))

18.848292603275663

In [9]:
poly_features = PolynomialFeatures(degree = 3,include_bias=False)
X_train_degree = poly_features.fit_transform(X_train)
X_test_degree =  poly_features.fit_transform(X_test)
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train_degree,y_train)
mean_squared_error(y_test,lin_reg.predict(X_test_degree))

18.805111358604705

> set.seed(2)

> train=sample (392,196)

> lm.fit=lm(mpg∼horsepower ,subset=train)

> mean((mpg -predict(lm.fit ,Auto))[-train]^2)

> lm.fit2=lm(mpg∼poly(horsepower ,2),data=Auto,subset=train)

> mean((mpg -predict(lm.fit2,Auto))[-train]^2)

> lm.fit3=lm(mpg∼poly(horsepower ,3),data=Auto,subset=train)

> mean((mpg -predict(lm.fit3,Auto))[-train]^2)


In [10]:
train_set, test_set = train_test_split(Auto, test_size = 0.5, random_state = 2)
y_train,X_train = patsy.dmatrices('mpg ~ 0 + horsepower',data = train_set,return_type="dataframe")
y_test,X_test = patsy.dmatrices('mpg ~ 0 + horsepower',data = test_set,return_type="dataframe")
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train,y_train)
mean_squared_error(y_test,lin_reg.predict(X_test))

23.442643969985735

In [11]:
poly_features = PolynomialFeatures(degree = 2,include_bias=False)
X_train_degree = poly_features.fit_transform(X_train)
X_test_degree =  poly_features.fit_transform(X_test)
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train_degree,y_train)
mean_squared_error(y_test,lin_reg.predict(X_test_degree))

18.550198801910312

In [12]:
poly_features = PolynomialFeatures(degree = 3,include_bias=False)
X_train_degree = poly_features.fit_transform(X_train)
X_test_degree =  poly_features.fit_transform(X_test)
lin_reg = linear_model.LinearRegression()
lin_reg.fit(X_train_degree,y_train)
mean_squared_error(y_test,lin_reg.predict(X_test_degree))

18.59522229455435

**使用statsmodels方案再次实现，为了和5.1.2的glm做个比较**

In [13]:
train_set, test_set = train_test_split(Auto, test_size = 0.5, random_state = 1)
lm_fit = smf.ols('mpg ~ horsepower',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

24.80212062059357

In [14]:
lm_fit = smf.ols('mpg ~ horsepower + I(horsepower ** 2)',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

18.848292603274604

In [15]:
lm_fit = smf.ols('mpg ~ horsepower + I(horsepower ** 2) + I(horsepower ** 3)',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

18.805111358430317

In [16]:
train_set, test_set = train_test_split(Auto, test_size = 0.5, random_state = 2)
lm_fit = smf.ols('mpg ~ horsepower',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

23.442643969985753

In [17]:
lm_fit = smf.ols('mpg ~ horsepower + I(horsepower ** 2)',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

18.550198801910696

In [18]:
lm_fit = smf.ols('mpg ~ horsepower + I(horsepower ** 2) + I(horsepower ** 3)',train_set).fit()
((test_set['mpg'] - lm_fit.predict(test_set['horsepower']))**2).mean()

18.595222294405293

**5.3.2 Leave-One-Out Cross-Validation**

> glm.fit=glm(mpg∼horsepower ,data=Auto)

> coef(glm.fit)

> lm.fit=lm(mpg∼horsepower ,data=Auto)

> coef(lm.fit)

In [19]:
lm_fit = smf.glm('mpg ~ horsepower',Auto).fit()
lm_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64

In [20]:
lm_fit = smf.ols('mpg ~ horsepower',Auto).fit()
lm_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64

> library(boot)

> glm.fit=glm(mpg∼horsepower ,data=Auto)

> cv.err=cv.glm(Auto ,glm.fit)

> cv.err$delta

In [21]:
from sklearn.model_selection import cross_val_score
y_mpg,X_horsepower_float = patsy.dmatrices('mpg ~ 0 + horsepower',data = Auto,return_type="dataframe")
-cross_val_score(lin_reg, X_horsepower_float, y_mpg, cv = len(y_mpg), scoring='neg_mean_squared_error').mean()

24.231513517929226

> cv.error=rep(0,5)

> for (i in 1:5){
+ glm.fit=glm(mpg∼poly(horsepower ,i),data=Auto)
+ cv.error[i]=cv.glm(Auto,glm.fit)$delta[1]
+ }

> cv.error

In [22]:
def msr_of_polynomial_cv(data,model,degree = 1,cv = 10,groups = None):
    lin_reg = linear_model.LinearRegression()
    y,X = patsy.dmatrices(model,data = data,return_type="dataframe")
    poly_features = PolynomialFeatures(degree = degree,include_bias=False)
    X_degree = poly_features.fit_transform(X)
    msr = -cross_val_score(lin_reg, X_degree, y, cv = cv, scoring='neg_mean_squared_error',groups = groups)
    return msr
for i in range(5):
    print((msr_of_polynomial_cv(Auto,'mpg ~ 0 + horsepower',degree = i + 1,cv = len(y_mpg))).mean().round(2))

24.23
19.25
19.33
19.42
19.03


**5.3.3 k-Fold Cross-Validation**

> set.seed(17)

> cv.error.10=rep(0,10)

> for (i in 1:10){
+ glm.fit=glm(mpg∼poly(horsepower ,i),data=Auto)
+ cv.error.10[i]=cv.glm(Auto,glm.fit,K=10)$delta[1]
+ }

> cv.error.10

In [23]:
np.random.seed(17)
#shuffled_indices = np.random.permutation(len(Auto))
for i in range(10):
    #print((msr_of_polynomial_cv(Auto.iloc[shuffled_indices],'mpg ~ horsepower',degree = i + 1,cv = 10)).mean().round(2))
    print((msr_of_polynomial_cv(Auto,'mpg ~ 0 + horsepower',degree = i + 1,cv = 10)).mean().round(2))

27.44
21.24
21.34
21.35
20.91
20.78
20.99
21.08
21.04
20.98


**5.3.4 The Bootstrap**

In [24]:
Portfolio = pd.read_csv("data/Portfolio.csv")
Portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


> alpha.fn=function (data,index){
+ X=data$X[index]
+ Y=data$Y[index]
+ return((var(Y)-cov(X,Y))/(var(X)+var(Y)-2*cov(X,Y)))
+ }

In [25]:
#根据自助法数据 α估计生成标准误差
def alpha_fn(data, index_start = 0,index_end = None):
    if index_end == None:
        index_end = len(data)
    X = data['X'].iloc[index_start:index_end]
    Y = data['Y'].iloc[index_start:index_end]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2* np.cov(X,Y)[0,1])
    #对于pd数据，可以使用return (Y.var() - X.cov(Y))/(X.var() + Y.var() - 2* X.cov(Y))

In [26]:
#> alpha.fn(Portfolio ,1:100)
alpha_fn(Portfolio)

0.5766511516104116

> set.seed(1)

> alpha.fn(Portfolio ,sample (100,100, replace=T))

In [27]:
alpha_fn(Portfolio.sample(100,replace=True,random_state= 1))

0.4504820492455901

In [28]:
#> boot(Portfolio ,alpha.fn,R=1000)定义对data根据statistic统计方式采样R次的生成数据，详见实验
def bootstrap_stats(data, statistic, R):
    n_shape = len(data)
    try:
        statistic_original = list(statistic(data))
    except TypeError:
        statistic_original = [statistic(data)]
    statistic_list = []
    train_list = []
    statistic_len = len(statistic_original)
    for statistic_t in range(statistic_len):
        statistic_list.append([])
    for random_state in range(R):
        train = data.sample(n = n_shape,replace=True,random_state= random_state)
        try:
            float(statistic(train))
        except:
            try: 
                for statistic_factor in list(statistic(train)):
                    float(statistic_factor)
            except:
                continue
        train_list.append(train)
        for statistic_t in range(statistic_len):
            try:
                statistic_list[statistic_t].append(list(statistic(train))[statistic_t])
            except TypeError:
                statistic_list[statistic_t].append([statistic(train)][statistic_t])
    statistic_index = []
    statistic_mean = []
    statistic_std_error = []
    statistic_bias = []
    for statistic_t in range(statistic_len):
        statistic_index.append('t' + str(statistic_t) + '*')
        statistic_mean.append(np.mean(statistic_list[statistic_t]))
        statistic_std_error.append(((np.sum((statistic_list[statistic_t] - statistic_mean[statistic_t]) ** 2)) / (R - 1)) ** 0.5)
        statistic_bias.append(statistic_original[statistic_t] - statistic_mean[statistic_t])
    bootstrap_stats_pd = pd.DataFrame({'original':statistic_original,
                                       'mean':statistic_mean,
                                       'bias':statistic_bias,
                                       'std. error':statistic_std_error                                      
                                      },
                                     index = statistic_index)
    print(bootstrap_stats_pd)
    return statistic_list
statistic_list = bootstrap_stats(Portfolio ,alpha_fn, R=1000)

     original      mean      bias  std. error
t0*  0.576651  0.574853  0.001798    0.092798


> boot.fn=function (data ,index)
+ return(coef(lm(mpg∼horsepower ,data=data,subset=index)))

> boot.fn(Auto ,1:392)

In [29]:
def boot_fn(data):
    return(smf.ols('mpg ~ horsepower',data).fit().params)
boot_fn(Auto)

Intercept     39.935861
horsepower    -0.157845
dtype: float64

> set.seed(1)

> boot.fn(Auto ,sample (392,392, replace=T))

In [30]:
boot_fn(Auto.sample(392,replace=True,random_state= 1))

Intercept     39.658479
horsepower    -0.155898
dtype: float64

In [31]:
boot_fn(Auto.sample(392,replace=True,random_state= 2))

Intercept     39.101022
horsepower    -0.151630
dtype: float64

In [32]:
#> boot(Auto ,boot.fn,1000)
statistic_list = bootstrap_stats(Auto, boot_fn, R=1000)

      original       mean      bias  std. error
t0*  39.935861  39.952723 -0.016862    0.862084
t1*  -0.157845  -0.158133  0.000288    0.007416


In [33]:
#> summary(lm(mpg∼horsepower ,data=Auto))$coef
smf.ols('mpg ~ horsepower',Auto).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Sun, 29 Dec 2019   Prob (F-statistic):           7.03e-81
Time:                        22:32:33   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
==============================================================================
Omnibus:                       16.432   Durbin-Watson:                   0.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.305
Skew:                           0.492   Prob(JB):                     0.000175
Kurtosis:                       3.299   Cond. No.                         322.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

> boot.fn=function (data ,index)
+ coefficients(lm(mpg∼horsepower +I(horsepower ^2),data=data ,
subset=index))

> set.seed(1)

> boot(Auto ,boot.fn,1000)


In [34]:
def boot_fn(data):
    return(smf.ols('mpg ~ horsepower + I( horsepower ** 2)',data).fit().params)
statistic_list = bootstrap_stats(Auto, boot_fn, R=1000)

      original       mean      bias  std. error
t0*  56.900100  56.934265 -0.034165    2.082847
t1*  -0.466190  -0.467016  0.000827    0.033048
t2*   0.001231   0.001234 -0.000004    0.000119


In [35]:
#summary(lm(mpg∼horsepower +I(horsepower ^2),data=Auto))$coef
smf.ols('mpg ~ horsepower + I( horsepower ** 2)',Auto).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Sun, 29 Dec 2019   Prob (F-statistic):           5.40e-99
Time:                        22:33:41   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             56.9001      1.800     31.604      0.000      53.360      60.440
horsepower            -0.4662      0.031    -14.978      0.000      -0.527      -0.405
I(horsepower ** 2)     0.0012      0.000     10.080      0.000       0.001       0.001
==============================================================================
Omnibus:                       16.158   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.662
Skew:                           0.218   Prob(JB):                     2.20e-07
Kurtosis:                       4.299   Cond. No.                     1.29e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""